In [4]:
import cv2
import numpy as np
from collections import deque
import os

class FightDetectorHolistic:
    def __init__(self):
        try:
            import mediapipe as mp
            self.mp = mp
            self.mp_holistic = mp.solutions.holistic
            self.mp_drawing = mp.solutions.drawing_utils
            self.mp_drawing_styles = mp.solutions.drawing_styles
            
            # Инициализация Holistic
            self.holistic = self.mp_holistic.Holistic(
                static_image_mode=False,
                model_complexity=1,
                smooth_landmarks=True,
                min_detection_confidence=0.7,
                min_tracking_confidence=0.7
            )
            print("✅ MediaPipe Holistic инициализирован успешно")
            
        except ImportError:
            print("❌ MediaPipe не установлен. Установите: pip install mediapipe")
            exit(1)
        except Exception as e:
            print(f"❌ Ошибка инициализации MediaPipe: {e}")
            exit(1)
        
        # Параметры для детекции драки
        self.fight_detected = False
        self.fight_start_frame = None
        self.fight_probability = 0
        self.consecutive_fight_frames = 0
        
        # История для анализа движений
        self.pose_history = deque(maxlen=10)
        
        # Пороговые значения
        self.VIOLENT_MOVEMENT_THRESHOLD = 0.1
        self.RAPID_HAND_MOVEMENT_THRESHOLD = 0.15
        self.FIGHT_CONFIRMATION_FRAMES = 10
        
        # Цвета для визуализации
        self.COLOR_SAFE = (0, 255, 0)      # Зеленый
        self.COLOR_WARNING = (0, 255, 255) # Желтый
        self.COLOR_DANGER = (0, 0, 255)    # Красный
    
    def calculate_distance(self, point1, point2):
        """Вычисляет расстояние между двумя точками"""
        return np.sqrt((point1.x - point2.x)**2 + (point1.y - point2.y)**2)
    
    def calculate_angle(self, a, b, c):
        """Вычисляет угол между тремя точками"""
        a = np.array([a.x, a.y])
        b = np.array([b.x, b.y])
        c = np.array([c.x, c.y])
        
        ba = a - b
        bc = c - b
        
        cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
        angle = np.arccos(np.clip(cosine_angle, -1.0, 1.0))
        
        return np.degrees(angle)
    
    def detect_aggressive_hand_movements(self, pose_landmarks):
        """Обнаруживает агрессивные движения рук"""
        if not pose_landmarks:
            return 0
        
        aggressive_score = 0
        
        try:
            # Ключевые точки для рук
            left_wrist = pose_landmarks.landmark[self.mp_holistic.PoseLandmark.LEFT_WRIST]
            right_wrist = pose_landmarks.landmark[self.mp_holistic.PoseLandmark.RIGHT_WRIST]
            left_elbow = pose_landmarks.landmark[self.mp_holistic.PoseLandmark.LEFT_ELBOW]
            right_elbow = pose_landmarks.landmark[self.mp_holistic.PoseLandmark.RIGHT_ELBOW]
            left_shoulder = pose_landmarks.landmark[self.mp_holistic.PoseLandmark.LEFT_SHOULDER]
            right_shoulder = pose_landmarks.landmark[self.mp_holistic.PoseLandmark.RIGHT_SHOULDER]
            
            # Углы рук для обнаружения ударов
            left_arm_angle = self.calculate_angle(left_shoulder, left_elbow, left_wrist)
            right_arm_angle = self.calculate_angle(right_shoulder, right_elbow, right_wrist)
            
            # Вытянутая рука (удар)
            if left_arm_angle > 150 or right_arm_angle > 150:
                aggressive_score += 2
            
            # Руки перед телом (агрессивная поза)
            nose = pose_landmarks.landmark[self.mp_holistic.PoseLandmark.NOSE]
            if (left_wrist.x < nose.x - 0.1 and left_wrist.y < left_shoulder.y) or \
               (right_wrist.x > nose.x + 0.1 and right_wrist.y < right_shoulder.y):
                aggressive_score += 1
                
        except Exception as e:
            print(f"Ошибка в детекции рук: {e}")
        
        return aggressive_score
    
    def detect_violent_leg_movements(self, pose_landmarks):
        """Обнаруживает агрессивные движения ног"""
        if not pose_landmarks:
            return 0
        
        leg_score = 0
        
        try:
            # Ключевые точки для ног
            left_hip = pose_landmarks.landmark[self.mp_holistic.PoseLandmark.LEFT_HIP]
            right_hip = pose_landmarks.landmark[self.mp_holistic.PoseLandmark.RIGHT_HIP]
            left_knee = pose_landmarks.landmark[self.mp_holistic.PoseLandmark.LEFT_KNEE]
            right_knee = pose_landmarks.landmark[self.mp_holistic.PoseLandmark.RIGHT_KNEE]
            left_ankle = pose_landmarks.landmark[self.mp_holistic.PoseLandmark.LEFT_ANKLE]
            right_ankle = pose_landmarks.landmark[self.mp_holistic.PoseLandmark.RIGHT_ANKLE]
            
            # Углы ног для обнаружения пинков
            left_leg_angle = self.calculate_angle(left_hip, left_knee, left_ankle)
            right_leg_angle = self.calculate_angle(right_hip, right_knee, right_ankle)
            
            # Вытянутая нога (пинок)
            if left_leg_angle > 160 or right_leg_angle > 160:
                leg_score += 2
            
            # Высоко поднятая нога
            if left_ankle.y < left_hip.y - 0.2 or right_ankle.y < right_hip.y - 0.2:
                leg_score += 1
                
        except Exception as e:
            print(f"Ошибка в детекции ног: {e}")
        
        return leg_score
    
    def analyze_body_posture(self, pose_landmarks):
        """Анализирует общую позу тела на агрессивность"""
        if not pose_landmarks:
            return 0
        
        posture_score = 0
        
        try:
            # Плечи и положение тела
            left_shoulder = pose_landmarks.landmark[self.mp_holistic.PoseLandmark.LEFT_SHOULDER]
            right_shoulder = pose_landmarks.landmark[self.mp_holistic.PoseLandmark.RIGHT_SHOULDER]
            left_hip = pose_landmarks.landmark[self.mp_holistic.PoseLandmark.LEFT_HIP]
            right_hip = pose_landmarks.landmark[self.mp_holistic.PoseLandmark.RIGHT_HIP]
            
            # Ширина стойки (агрессивная стойка)
            shoulder_width = abs(left_shoulder.x - right_shoulder.x)
            if shoulder_width > 0.2:
                posture_score += 1
            
            # Наклон тела вперед (атакующая поза)
            nose = pose_landmarks.landmark[self.mp_holistic.PoseLandmark.NOSE]
            shoulder_center_y = (left_shoulder.y + right_shoulder.y) / 2
            hip_center_y = (left_hip.y + right_hip.y) / 2
            
            if nose.y < shoulder_center_y and shoulder_center_y < hip_center_y:
                posture_score += 1
                
        except Exception as e:
            print(f"Ошибка в анализе позы: {e}")
        
        return posture_score
    
    def detect_rapid_movements(self, current_pose, previous_pose):
        """Обнаруживает резкие движения между кадрами"""
        if not current_pose or not previous_pose:
            return 0
        
        movement_score = 0
        
        try:
            # Ключевые точки для отслеживания движения
            key_points = [
                self.mp_holistic.PoseLandmark.LEFT_WRIST,
                self.mp_holistic.PoseLandmark.RIGHT_WRIST,
                self.mp_holistic.PoseLandmark.LEFT_ANKLE,
                self.mp_holistic.PoseLandmark.RIGHT_ANKLE
            ]
            
            for point in key_points:
                current_point = current_pose.landmark[point]
                previous_point = previous_pose.landmark[point]
                
                movement = self.calculate_distance(current_point, previous_point)
                if movement > self.RAPID_HAND_MOVEMENT_THRESHOLD:
                    movement_score += 1
                    
        except Exception as e:
            print(f"Ошибка в детекции движений: {e}")
        
        return movement_score
    
    def analyze_frame(self, results, previous_pose):
        """Анализирует кадр и возвращает вероятность драки и индикаторы"""
        fight_probability = 0
        indicators = {
            'aggressive_hands': 0,
            'aggressive_legs': 0,
            'aggressive_posture': 0,
            'rapid_movements': 0,
            'people_count': 0
        }
        
        if results.pose_landmarks:
            indicators['people_count'] = 1
            
            # Анализ агрессивных индикаторов
            indicators['aggressive_hands'] = self.detect_aggressive_hand_movements(results.pose_landmarks)
            indicators['aggressive_legs'] = self.detect_violent_leg_movements(results.pose_landmarks)
            indicators['aggressive_posture'] = self.analyze_body_posture(results.pose_landmarks)
            
            # Анализ резких движений
            if previous_pose:
                indicators['rapid_movements'] = self.detect_rapid_movements(results.pose_landmarks, previous_pose)
            
            # Расчет общей вероятности драки
            fight_probability = min(100, (
                indicators['aggressive_hands'] * 20 +
                indicators['aggressive_legs'] * 20 +
                indicators['aggressive_posture'] * 15 +
                indicators['rapid_movements'] * 25
            ))
        
        return fight_probability, indicators
    
    def draw_detection_info(self, frame, fight_probability, indicators, frame_count):
        """Рисует информацию о детекции на кадре"""
        height, width = frame.shape[:2]
        
        # Определяем цвет и статус
        if fight_probability > 70:
            color = self.COLOR_DANGER
            status = "FIGHT was detected!"
            alert_level = "high"
        elif fight_probability > 40:
            color = self.COLOR_WARNING
            status = "potential fight"
            alert_level = "middle"
        else:
            color = self.COLOR_SAFE
            status = "normal"
            alert_level = "low"
        
        # Основной индикатор
        cv2.putText(frame, f"status: {status}", (20, 40),
                   cv2.FONT_HERSHEY_SIMPLEX, 1, color, 3)
        
        cv2.putText(frame, f"Probability: {fight_probability}%", (20, 80),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
        
        cv2.putText(frame, f"level: {alert_level}", (20, 110),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2)
        
        # Детальная информация
        y_offset = 150
        details = [
            f"agressive hand: {indicators['aggressive_hands']}",
            f"agressive legs: {indicators['aggressive_legs']}",
            f"agressive pose: {indicators['aggressive_posture']}",
            f"rapid movements: {indicators['rapid_movements']}",
            f"people: {indicators['people_count']}"
        ]
        
        for i, detail in enumerate(details):
            cv2.putText(frame, detail, (20, y_offset + i * 25),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 1)
        
        # Визуальное предупреждение при высокой вероятности
        if fight_probability > 70:
            cv2.rectangle(frame, (0, 0), (width, height), color, 10)
            cv2.putText(frame, "fight! fight!", (width//2 - 200, height//2),
                       cv2.FONT_HERSHEY_SIMPLEX, 2, color, 4)
        
        # Номер кадра
        cv2.putText(frame, f"frame: {frame_count}", (width - 150, 30),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)
    
    def draw_landmarks(self, frame, results):
        """Рисует landmarks на кадре"""
        try:
            if results.pose_landmarks:
                self.mp_drawing.draw_landmarks(
                    frame,
                    results.pose_landmarks,
                    self.mp_holistic.POSE_CONNECTIONS,
                    landmark_drawing_spec=self.mp_drawing_styles.get_default_pose_landmarks_style()
                )
            
            if results.left_hand_landmarks:
                self.mp_drawing.draw_landmarks(
                    frame,
                    results.left_hand_landmarks,
                    self.mp_holistic.HAND_CONNECTIONS,
                    landmark_drawing_spec=self.mp_drawing_styles.get_default_hand_landmarks_style()
                )
            
            if results.right_hand_landmarks:
                self.mp_drawing.draw_landmarks(
                    frame,
                    results.right_hand_landmarks,
                    self.mp_holistic.HAND_CONNECTIONS,
                    landmark_drawing_spec=self.mp_drawing_styles.get_default_hand_landmarks_style()
                )
        except Exception as e:
            print(f"Ошибка при отрисовке landmarks: {e}")
    
    def process_video(self, video_path, output_path=None):
        """Обрабатывает видео и детектирует драки"""
        cap = cv2.VideoCapture(video_path)
        
        if not cap.isOpened():
            print(f"❌ Не удалось открыть видео: {video_path}")
            return
        
        # Получаем параметры видео
        fps = cap.get(cv2.CAP_PROP_FPS)
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        
        # Создаем VideoWriter для выходного видео
        if output_path:
            fourcc = cv2.VideoWriter_fourcc(*'mp4v')
            out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
        else:
            out = None
        
        previous_pose = None
        frame_count = 0
        
        print("🚀 Запуск детектора драк с MediaPipe Holistic...")
        print("⏹️  Нажмите 'q' для выхода")
        print("⏸️  Нажмите 'p' для паузы")
        
        while cap.isOpened():
            ret, frame = cap.read()
            if not ret:
                break
            
            frame_count += 1
            
            # Конвертируем BGR в RGB
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame_rgb.flags.writeable = False
            
            # Детекция с помощью Holistic
            results = self.holistic.process(frame_rgb)
            
            frame_rgb.flags.writeable = True
            frame_bgr = cv2.cvtColor(frame_rgb, cv2.COLOR_RGB2BGR)
            
            # Анализ кадра
            fight_prob, indicators = self.analyze_frame(results, previous_pose)
            
            # Рисуем landmarks
            self.draw_landmarks(frame_bgr, results)
            
            # Рисуем информацию о детекции
            self.draw_detection_info(frame_bgr, fight_prob, indicators, frame_count)
            
            # Проверка начала драки
            if fight_prob > 70:
                self.consecutive_fight_frames += 1
                if (self.consecutive_fight_frames >= self.FIGHT_CONFIRMATION_FRAMES and 
                    not self.fight_detected):
                    self.fight_detected = True
                    self.fight_start_frame = frame_count
                    print(f"⚠️  fight was detected on frame {frame_count}!")
                    print(f"   - probability: {fight_prob}%")
            else:
                self.consecutive_fight_frames = max(0, self.consecutive_fight_frames - 1)
            
            # Отображаем кадр
            cv2.imshow('Fight Detector - MediaPipe Holistic', frame_bgr)
            
            # Сохраняем кадр если нужно
            if out:
                out.write(frame_bgr)
            
            # Обновляем историю
            previous_pose = results.pose_landmarks
            
            # Управление
            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                break
            elif key == ord('p'):
                print("⏸️  Пауза. Нажмите любую клавишу для продолжения...")
                cv2.waitKey(0)
        
        # Освобождаем ресурсы
        cap.release()
        if out:
            out.release()
        cv2.destroyAllWindows()
        
        # Итоговая статистика
        self.print_final_stats(frame_count)
    
    def print_final_stats(self, total_frames):
        """Выводит итоговую статистику"""
        print("\n" + "="*50)
        print("📊 ИТОГОВАЯ СТАТИСТИКА ДЕТЕКЦИИ ДРАК")
        print("="*50)
        
        if self.fight_detected and self.fight_start_frame:
            start_time = self.fight_start_frame / 30
            print(f"✅ Драка обнаружена на кадре: {self.fight_start_frame}")
            print(f"⏱️  Время начала: {start_time:.1f} секунд")
        else:
            print("❌ Драка не обнаружена")
        
        print(f"🎞️  Всего обработано кадров: {total_frames}")



detector = FightDetectorHolistic()

video_path = "data/input/1.mp4"  
output_path = "data/output/1_detection.mp4"  


if output_path:
    os.makedirs(os.path.dirname(output_path), exist_ok=True)

detector.process_video(video_path, output_path)

✅ MediaPipe Holistic инициализирован успешно
🚀 Запуск детектора драк с MediaPipe Holistic...
⏹️  Нажмите 'q' для выхода
⏸️  Нажмите 'p' для паузы
⚠️  fight was detected on frame 421!
   - probability: 95%

📊 ИТОГОВАЯ СТАТИСТИКА ДЕТЕКЦИИ ДРАК
✅ Драка обнаружена на кадре: 421
⏱️  Время начала: 14.0 секунд
🎞️  Всего обработано кадров: 2281
